In [1]:
import cv2
import time
import tkinter as tk
from tkinter import filedialog

# Define the region of interest (ROI)
roi_top = 90
roi_bottom = 125

# Initialize variables for speed calculation
start_time = time.time()
last_position = None

# Function to open a file dialog and get the selected file path
def browse_file():
    file_path = filedialog.askopenfilename()
    if file_path:
        entry_var.set(file_path)

# Function to start processing the video
# Function to start processing the video
def start_processing(roi_top, roi_bottom):
    video_path = entry_var.get()

    if video_path:
        # Video capture
        cap = cv2.VideoCapture(video_path)

        # Background subtractor
        bg_subtractor = cv2.createBackgroundSubtractorMOG2()

        # Initialize last_position
        last_position = None

        while True:
            ret, frame = cap.read()

            if not ret:
                break

            # Extract the region of interest (ROI)
            roi = frame[roi_top:roi_bottom, :]

            # Apply background subtraction
            fg_mask = bg_subtractor.apply(roi)

            # Threshold the foreground mask
            _, thresh = cv2.threshold(fg_mask, 200, 255, cv2.THRESH_BINARY)

            # Find contours in the thresholded image
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Draw bounding boxes around detected objects (cars) and display speed
            for contour in contours:
                x, y, w, h = cv2.boundingRect(contour)

                # Filter out small contours (noise)
                if cv2.contourArea(contour) > 100:
                    cv2.rectangle(frame, (x, roi_top + y), (x + w, roi_top + y + h), (0, 255, 0), 2)

                    # Calculate speed based on position change
                    current_position = (x + w / 2, roi_top + y + h / 2)
                    if last_position is not None:
                        # Convert speed to km/h based on the physical dimensions of your scene
                        speed = abs(current_position[0] - last_position[0]) / (time.time() - start_time) * 30 * 0.001 * 3600
                        # Adjust scale as needed

                        # Display speed above the car
                        cv2.putText(frame, f"Speed: {speed:.2f} km/h", (x, roi_top + y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

                    last_position = current_position

            # Display the result
            cv2.imshow('Video', frame)

            if cv2.waitKey(33) == 27:
                break

        # Release resources
        cap.release()
        cv2.destroyAllWindows()

# Create the main window
root = tk.Tk()
root.title("Car Speed Detection")

# Entry widget to display the selected file path
entry_var = tk.StringVar()
entry = tk.Entry(root, textvariable=entry_var, width=40)
entry.grid(row=0, column=0, padx=10, pady=10, columnspan=2)

# Button to browse and select a video file
browse_button = tk.Button(root, text="Browse", command=browse_file)
browse_button.grid(row=0, column=2, padx=10, pady=10)

# Button to start processing the selected video
start_button = tk.Button(root, text="Start Processing", command=lambda: start_processing(roi_top, roi_bottom))
start_button.grid(row=1, column=0, columnspan=3, pady=10)

# Main loop
root.mainloop()